# Introduction

Goal

The goal of this project is to act as a Data Analyst for the Google Merchandise Store, and analyze Google Analytics data from Bigquery using SQL. The period for this analysis will be 2016/8/1 - 2017/8/1.

Site: https://shop.googlemerchandisestore.com/

For this analysis we'll focus on 3 main goals:

- Understand the composition of current site traffic
- Understand the flow and conversion path of users
- Forecast product demand

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [4]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [6]:
# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')
# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [7]:
[x.table_id for x in client.list_tables(dataset)][:5]

['ga_sessions_20160801',
 'ga_sessions_20160802',
 'ga_sessions_20160803',
 'ga_sessions_20160804',
 'ga_sessions_20160805']

In [8]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  channelGrouping       5 non-null      object
 14  socialEngagementType  5 non-null      object
dtypes: Int64(4), object(11)
memory usage: 752.0+

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [9]:
# Get format schema nested field

def format_schema_field(schema_field, indent=0):
    indent_str = "  " * indent
    field_info = f"{indent_str}{schema_field.name} ({schema_field.field_type})"
    
    if schema_field.mode != "NULLABLE":
        field_info += f" - {schema_field.mode}"
    
    if schema_field.description:
        field_info += f" - {schema_field.description}"
    
    nested_indent = indent + 2
    if schema_field.field_type == "RECORD":
        for sub_field in schema_field.fields:
            field_info += "\n" + format_schema_field(sub_field, nested_indent)
    
    return field_info


table_ref = dataset_ref.table('ga_sessions_20160801')
table = client.get_table(table_ref)

# Display schemas
print("SCHEMA field for the 'totals' column:\n")
print(format_schema_field(table.schema[5]))
print()

print("\nSCHEMA field for the 'trafficSource' column:\n")
print(format_schema_field(table.schema[6]))
print()

print("\nSCHEMA field for the 'device' column:\n")
print(format_schema_field(table.schema[7]))
print()

print("\nSCHEMA field for the 'geoNetwork' column:\n")
print(format_schema_field(table.schema[8]))
print()

print("\nSCHEMA field for the 'customDimensions' column:\n")
print(format_schema_field(table.schema[9]))
print()

print("\nSCHEMA field for the 'hits' column:\n")
print(format_schema_field(table.schema[10]))

SCHEMA field for the 'totals' column:

totals (RECORD)
    visits (INTEGER)
    hits (INTEGER)
    pageviews (INTEGER)
    timeOnSite (INTEGER)
    bounces (INTEGER)
    transactions (INTEGER)
    transactionRevenue (INTEGER)
    newVisits (INTEGER)
    screenviews (INTEGER)
    uniqueScreenviews (INTEGER)
    timeOnScreen (INTEGER)
    totalTransactionRevenue (INTEGER)
    sessionQualityDim (INTEGER)


SCHEMA field for the 'trafficSource' column:

trafficSource (RECORD)
    referralPath (STRING)
    campaign (STRING)
    source (STRING)
    medium (STRING)
    keyword (STRING)
    adContent (STRING)
    adwordsClickInfo (RECORD)
        campaignId (INTEGER)
        adGroupId (INTEGER)
        creativeId (INTEGER)
        criteriaId (INTEGER)
        page (INTEGER)
        slot (STRING)
        criteriaParameters (STRING)
        gclId (STRING)
        customerId (INTEGER)
        adNetworkType (STRING)
        targetingCriteria (RECORD)
            boomUserlistId (INTEGER)
        isV

**Most Popular Pages**

A high bounce rate can indicate either:

1. Users are not seeing what they expect when they enter the site, and Search Engine Optimization (SEO) needs to be done.
2. There is a functionality or display issue with the landing page.

In [47]:
query = """
SELECT
  SUBSTRING(hit.page.pagePath, INSTR(hit.page.pagePath, '/', -1) + 1) page,
  COUNT(*) views,
  SUM(totals.bounces) / COUNT(*) bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND hit.type = 'PAGE'
GROUP BY page
ORDER BY views DESC
"""

df = client.query(query).to_dataframe()

df.head(20)

,page,views,bounce_rate
0,home,1036716,0.297107
1,quickview,359876,0.000270
2,basket.html,209379,0.020537
3,youtube,145049,0.316190
4,signin.html,101299,0.058322
5,store.html,93552,0.025505
6,men++s+t+shirts,67472,0.135389
7,asearch.html,62380,0.031677
8,electronics,56854,0.047121
9,apparel,56707,0.051122


In [46]:
fig = px.bar(df.head(20).sort_values(by='bounce_rate'), y='page', x='bounce_rate',
            title='Bounce Rate from Various Channel',
            labels={'bounce_rate': 'Bounce Rate',
                    'page': 'Page'},
            orientation='h')

fig.show()

NOTES:

Looking at the bounce rates, it looks like Men's Apparel and Bags are comparatively lower than the other pages.

We can also see that the YouTube and Drinkware pages have a lot of views, but have a high bounce rate, indicating either a problem with the page's functionality, or users not seeing what they were expecting to when they entered the site.

Formulated the following hypotheses:

- The reason that YouTube is the second most visited landing page and yet has the highest bounce rate is that there are a large number of visitors looking for YouTuber merchandise, and not YouTube brand merchandise.
- The reason for login and Basket's high visits and low bounce rate is due to a large number of repeat visitors.
- Within the top 10 landing pages 3 are related to Men's Apparel, indicating it is an especially popular category

**Sessions across Browsers and Devices**

It is needed to see if there are any compatibility issues.

In [57]:
query = """
SELECT
  device.Browser browser,
  COUNT(*) sessions,
  SUM(totals.bounces) / COUNT(*) bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND hit.type = 'PAGE'
GROUP BY browser
ORDER BY sessions DESC
"""

df = client.query(query).to_dataframe()
df.head(10)

,browser,sessions,bounce_rate
0,Chrome,2653448,0.109769
1,Safari,544508,0.180686
2,Firefox,114184,0.195675
3,Internet Explorer,53988,0.218067
4,Edge,32460,0.175416
5,Android Webview,21934,0.204568
6,Safari (in-app),17016,0.236777
7,Opera,13210,0.243528
8,Opera Mini,11981,0.335281
9,UC Browser,4673,0.350096


In [58]:
fig = px.bar(df.head(10).sort_values(by='bounce_rate'), x='browser', y='bounce_rate',
            title='Bounce Rate from Various Region',
            labels={'bounce_rate': 'Bounce Rate',
                    'browser': 'Browser'})

fig.show()

In [59]:
fig = px.bar(df.head(10), x='browser', y='sessions',
            title='Sessions from Various Region',
            labels={'sessions': 'Sessions',
                    'browser': 'Browser'})

fig.show()

NOTES:
- The majority of sessions are from Chrome, but there are a significant number of sessions from Safari and Firefox as well.
- The bounce rate for Safari and Firefox is significantly higher than Chrome, which could indicate compatibility issues
- Among the top 5 browsers, the bounce rate for Chrome is comparatively low at 9%, while the other browsers are roughly 7% - 8% higher with the exception of Internet Explorer, having an bounce rate of 19%.

In [60]:
query = """
SELECT
  device.deviceCategory device,
  COUNT(*) sessions,
  SUM(totals.bounces) / COUNT(*) bounce_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND hit.type = 'PAGE'
GROUP BY device
ORDER BY sessions DESC
"""

df = client.query(query).to_dataframe()
df.head()

,device,sessions,bounce_rate
0,desktop,2642573,0.123770
1,mobile,713646,0.151966
2,tablet,122247,0.123586


In [62]:
fig = px.bar(df.sort_values(by='bounce_rate'), x='device', y='bounce_rate',
            title='Bounce Rate from Various Device',
            labels={'bounce_rate': 'Bounce Rate',
                    'device': 'Device'})

fig.show()

In [63]:
fig = px.bar(df, x='device', y='sessions',
            title='Sessions from Various Region',
            labels={'sessions': 'Sessions',
                    'device': 'Device'})

fig.show()

NOTES:
- The majority of sessions are on desktop, with a significant number also coming from mobile.

- We can see that Mobile is about 2.5% higher than other devices. It's possible that the site is not optimized for mobile viewing.

- The guessed factors:
> 1. The site not being optimized for mobile devices
> 2. Users first browsing product pages on their mobile devices, and then later returning on desktop to complete their purchase

It may therefore be useful to conduct a user test for mobile devices. With this we can understand where users may face problems not just on a browser and device level, but on a page, form, and even button level.

**Traffic Type and Quality**

In [65]:
query = """
SELECT
  trafficSource.medium medium,
  COUNT(*) sessions,
  SUM(totals.bounces) / COUNT(*) bounce_rate,
  SUM(totals.transactions) transactions,
  SUM(totals.totalTransactionRevenue) / 1000000 total_revenue_permillion,
  SUM(totals.transactions) / COUNT(*) conversion_rate
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h, UNNEST(h.hits) hit
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY medium
ORDER BY
  sessions DESC
"""

df = client.query(query).to_dataframe()
df.head(10)

,medium,sessions,bounce_rate,transactions,total_revenue_permillion,conversion_rate
0,(none),2246040,0.069684,363228,69842914.68,0.161719
1,organic,1141113,0.101857,97332,19504702.25,0.085296
2,referral,580309,0.285806,22040,3501224.27,0.037980
3,cpc,88591,0.055028,9457,1528330.72,0.106749
4,affiliate,55860,0.157698,334,29118.69,0.005979
5,cpm,41253,0.054372,5951,8676307.66,0.144256
6,(not set),509,0.108055,17,203.83,0.033399


In [66]:
fig = px.bar(df.sort_values(by='conversion_rate'), x='medium', y='conversion_rate',
            title='Conversion Rate from Various Medium',
            labels={'conversion_rate': 'Conversion Rate',
                    'medium': 'Medium'})

fig.show()

In [67]:
fig = px.bar(df.sort_values(by='total_revenue_permillion'), x='medium', y='total_revenue_permillion',
            title='Total Revenue per Million from Various Medium',
            labels={'total_revenue_permillion': 'Total Revenue per Million',
                    'medium': 'Medium'})

fig.show()

NOTES:
- It looks like the most traffic is either Organic or Referral
- The conversion rate for CPC and CPM is quite high compared to other sources.
- The conversion rate refers to the percentage of sessions that resulted in a transaction being completed.

**Identify Conversion Path and Possible Bottlenecks**

In [21]:
query = """
SELECT
  CASE 
    WHEN hit.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
    WHEN hit.eCommerceAction.action_type = '2' THEN 'Product detail views'
    WHEN hit.eCommerceAction.action_type = '5' THEN 'Check out'
    WHEN hit.eCommerceAction.action_type = '6' THEN 'Completed purchase'
  END action,
  COUNT(*) users
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
  UNNEST(h.hits) hit,
  UNNEST(hit.product) product
WHERE 
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
  AND
    (
      hit.eCommerceAction.action_type != '0'
      AND
      hit.eCommerceAction.action_type != '3'
      AND
      hit.eCommerceAction.action_type != '4'
    )
    GROUP BY action
    ORDER BY users DESC
"""

df = client.query(query).to_dataframe()
df.head()

,action,users
0,Click through of product lists,445760
1,Product detail views,362607
2,Check out,248936
3,Completed purchase,74457


NOTES:

In this query:

- The UNNEST(hit.product) operation will flatten the nested product array within each hit.
- This means for each hit, the number of rows generated will be multiplied by the number of products in the hit.product array.
- As a result, if a hit contains multiple products, it will contribute multiple rows to the aggregation.

In [22]:
fig = go.Figure(go.Funnel(
    y = df['action'],
    x = df['users'],
    textposition='inside',
    textinfo = "value+percent initial"
))

fig.update_layout(
    title="Google Merchandise Store Conversion Path",
    showlegend=False
)

# Show the plot
fig.show()

NOTES:

- 69% of users who viewed the product details proceeded to the check out page.

- We can see however that of those users, only 30% actually completed a purchase.

- May be there is a problem with the purchase form.

- May be users are browsing and tentatively deciding on the purchase, and then completing it in a later session

In [24]:
query = """ 
        SELECT
            product.v2ProductCategory AS category,
            SUM(totals.transactions) AS transactions,
            SUM(totals.totalTransactionRevenue)/1000000 AS total_revenue
        FROM 
            `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
            UNNEST(hits) AS hits,
            UNNEST(hits.product) AS product
        WHERE
            _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
        GROUP BY
            category
        ORDER BY
            total_revenue DESC
        """

df = client.query(query).to_dataframe()
df.head(20)

,category,transactions,total_revenue
0,(not set),155239,51801830.97
1,Apparel,133504,44923859.06
2,Home/Apparel/Men's/Men's-T-Shirts/,147131,25222312.08
3,Home/Office/,100994,20222511.87
4,Home/Electronics/,62603,13716689.08
5,Office,41837,12733564.97
6,Home/Apparel/,61127,11863650.76
7,Home/Apparel/Women's/Women's-T-Shirts/,58134,11448008.84
8,Home/Drinkware/Water Bottles and Tumblers/,44300,10600034.98
9,Home/Apparel/Men's/,77731,9781940.51


In [34]:
# Forecaset demand Men's T-Shirts


query = """ 
WITH daily_mens_tshirt_transactions AS
    (
    SELECT 
        date,
        SUM(totals.transactions) AS transactions
    FROM 
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
        UNNEST(hits) AS hits,
        UNNEST(hits.product) AS product
    WHERE
        _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND
        product.v2ProductCategory = "Home/Apparel/Men's/Men's-T-Shirts/"
    GROUP BY
        date
    ORDER BY
        date
    )
    SELECT
        date,
        AVG(transactions)
        OVER (
            ORDER BY date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING    
            ) AS avg_transactions
    FROM 
        daily_mens_tshirt_transactions
"""

df = client.query(query).to_dataframe()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# # Create the line plot
fig = px.line(df, x='date', y='avg_transactions', markers=True,
              title='Men\'s T-Shirts Conversions 7-Day Moving Average',
              labels={'date': 'Date', 'avg_transactions': 'Average Transaction'})

# Show the plot
fig.show()

Looking at the chart above, it would seem there is some seasonality in the demand for Men's T-shirts.

In [38]:
# Forecaset demand Men's T-Shirts


query = """ 
WITH daily_drinkware_transactions AS
    (
    SELECT 
        date,
        SUM(totals.transactions) AS transactions
    FROM 
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
        UNNEST(hits) AS hits,
        UNNEST(hits.product) AS product
    WHERE
        _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND
        product.v2ProductCategory = "Home/Drinkware/Water Bottles and Tumblers/"
    GROUP BY
        date
    ORDER BY
        date
    )
    SELECT
        date,
        AVG(transactions)
        OVER (
            ORDER BY date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING    
            ) AS avg_transactions
    FROM 
        daily_drinkware_transactions
"""

df = client.query(query).to_dataframe()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# # Create the line plot
fig = px.line(df, x='date', y='avg_transactions', markers=True,
              title='Drinkware Conversions 7-Day Moving Average',
              labels={'date': 'Date', 'avg_transactions': 'Average Transaction'})

# Show the plot
fig.show()

In [37]:
query = """
WITH daily_electronics_transactions AS
    (
    SELECT 
        date,
        SUM(totals.transactions) AS transactions
    FROM 
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
        UNNEST(hits) AS hits,
        UNNEST(hits.product) AS product
    WHERE
        _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND
        product.v2ProductCategory = "Home/Electronics/"
    GROUP BY
        date
    ORDER BY
        date
    )
    SELECT
        date,
        AVG(transactions)
        OVER (
            ORDER BY date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING    
            ) AS avg_transactions
    FROM 
        daily_electronics_transactions

"""


df = client.query(query).to_dataframe()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# # Create the line plot
fig = px.line(df, x='date', y='avg_transactions', markers=True,
              title='Electronics Conversions 7-Day Moving Average',
              labels={'date': 'Date', 'avg_transactions': 'Average Transaction'})

# Show the plot
fig.show()

In [39]:
query = """
WITH daily_office_transactions AS
    (
    SELECT 
        date,
        SUM(totals.transactions) AS transactions
    FROM 
        `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
        UNNEST(hits) AS hits,
        UNNEST(hits.product) AS product
    WHERE
        _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
    AND
        product.v2ProductCategory = "Home/Office/"
    GROUP BY
        date
    ORDER BY
        date
    )
    SELECT
        date,
        AVG(transactions)
        OVER (
            ORDER BY date
            ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING    
            ) AS avg_transactions
    FROM 
        daily_office_transactions

"""


df = client.query(query).to_dataframe()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# # Create the line plot
fig = px.line(df, x='date', y='avg_transactions', markers=True,
              title='Office Conversions 7-Day Moving Average',
              labels={'date': 'Date', 'avg_transactions': 'Average Transaction'})

# Show the plot
fig.show()

NOTES:

The average transactions for August 2016 are very high compared to later periods, but since we don't know the cause of this we will ignore it for now.

Office and Electronics products seem to have steady demand year-round, Drinkware seems to see a spike in demand in December and March, and demand for Men's T-Shirts seem to increase in September, March, and August.

Source: https://www.kaggle.com/code/dillonmyrick/sql-eda-of-google-analytics-data#3.-Explore-and-Analyze-Data